In [1]:
# do a hacky import since brainaudio requires python 3.12 and the .wfst env requires an older
# python version 
from wfst_utils import build_lm_decoder, lm_decode, arrange_logits, _cer_and_wer
import sys
import numpy as np
import pandas as pd
import re
import pickle

ModuleNotFoundError: No module named 'lm_decoder'

In [22]:
load_lm = True

In [27]:
transcripts_24_cleaned = pd.read_pickle("/data2/brain2text/b2t_24/transcripts_val_cleaned.pkl")

In [5]:
lmDir = "/data2/brain2text/lm/languageModel"
acoustic_scale = 0.8
nbest = 1
return_n_best = False
rescore = False
nbest = 1

if load_lm and 'ngramDecoder' not in globals(): 
    
    ngramDecoder = build_lm_decoder(
        lmDir,
        acoustic_scale=acoustic_scale, #1.2
        nbest=nbest,
        beam=18
    )
    print("loaded LM")

loaded LM


I1103 12:58:53.529997 3650960 brain_speech_decoder.h:52] Reading fst /data2/brain2text/lm/languageModel/TLG.fst
I1103 13:01:30.777148 3650960 brain_speech_decoder.h:81] Reading symbol table /data2/brain2text/lm/languageModel/words.txt


In [24]:
logits_all = np.load("/data2/brain2text/b2t_24/logits/tm_transformer_24/logits_val.npz")

In [25]:
decoded_text_outputs = []
blank_penalty = np.log(2)
for logits in logits_all.values():
    
    rearranged_logits = arrange_logits(logits)
    
    decoded_text = lm_decode(
                    ngramDecoder,
                    rearranged_logits[0],
                    blankPenalty=blank_penalty,
                    returnNBest=return_n_best,
                    rescore=rescore,
                )
    
    decoded_text_outputs.append(decoded_text.strip())
    
_cer_and_wer(decoded_text_outputs, transcripts_24_cleaned)

(0.12166333757036499, 0.17312238588834333)

In [12]:
save_path = "/data2/brain2text/b2t_24/wfst_txt/tm_transformer_b2t_24+25_large_wide_bidir_grad_clip_cosine_decay.txt"
with open(save_path, mode="w") as file:
    for line in decoded_text_outputs:
        file.write(line + "\n")

In [ ]:
(0.10899152971838123, 0.17166757592289508)